## **Compute Correlation - PAC**

Algorithm 2
```
subjects = 1:40;
health = ["HC","PDoff","PDon"];
stims = ["sham","stim7","stim8"];
trials = 1:10;
pacs = ["Delta_Theta","Delta_Alhpa","Delta_Beta","Delta_Gamma","Theta_Alhpa","Theta_Beta","Theta_Gamma","Alhpa_Beta","Alhpa_Gamma","Beta_Gamma"];

datatable = [];

for subjectIdx = 1:length(subjects)
    % Over stim conditions only within 
    for stimIdx = 1:length(stims)
        % over trials within and between
        for trialIdx1 = 1:length(trials)
            for trialIdx2 = trialIdx1:length(trials)
                % over PACs within and between
                for pacIdx1 = 1:length(pacs)
                    for pacIdx2 = pacIdx1:length(pacs)
    					value = corr(pacIdx1 of trialId1, pacIdx1 of trialId1);
    					condition = health(this subjectIdx);
                        value = 1;
                        subject = subjects(subjectIdx);
                        stim = stims(stimIdx);
                        PAC1 = pacs(pacIdx1);
                        PAC2 = pacs(pacIdx2);
                        trial1 = trials(trialIdx1);
                        trial2 = trials(trialIdx2);
                        datatable = cat(1,datatable,table(subject,stim,trial1,trial2,PAC1,PAC2,value));
                    end
                end
            end
        end
    end
end
      
```


In [ ]:
# from itertools import combinations
from itertools import combinations_with_replacement

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import clear_output
from scipy.signal import resample
from scipy.stats import pearsonr

In [ ]:
## Merging all data to a single file

path_pacs_dataset = "/path-to-pacs-dir/"
df1 = pd.read_parquet(path_pacs_dataset + "data_1.parquet")
df2 = pd.read_parquet(path_pacs_dataset + "data_2.parquet")
df3 = pd.read_parquet(path_pacs_dataset + "data_3.parquet")
df4 = pd.read_parquet(path_pacs_dataset + "data_4.parquet")
df5 = pd.read_parquet(path_pacs_dataset + "data_5.parquet")
df6 = pd.read_parquet(path_pacs_dataset + "data_6.parquet")
df7 = pd.read_parquet(path_pacs_dataset + "data_7.parquet")
df8 = pd.read_parquet(path_pacs_dataset + "data_8.parquet")
df9 = pd.read_parquet(path_pacs_dataset + "data_9.parquet")

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9]).reset_index(inplace=True, drop=True)
table = pa.Table.from_pandas(df)
pq.write_table(table, path_pacs_dataset + 'data_merged.parquet', compression='BROTLI')

In [ ]:
## Load the dataset

df = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
df.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
df = df.dropna(subset=["peak_time", "reaction_time"])
print(df.info())
print(df)

print(df['medication'].unique())
df['medication'].replace('pdoffmed_gvs8', 'pdoffmed', inplace=True)
df['medication'].replace('hcoffmed_gvs8', 'hcoffmed', inplace=True)
df['medication'].replace('pdonmed_gvs8', 'pdonmed', inplace=True)
print(df['medication'].unique())

In [ ]:
## Find the region of interests

df["beta_gamma"]  = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 16:].ravel())
df["alpha_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 8:16].ravel())
df["theta_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 4:8].ravel())
df["delta_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, :4].ravel())

df["alpha_beta"]  = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, 8:16].ravel())
df["theta_beta"]  = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, 4:8].ravel())
df["delta_beta"]  = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, :4].ravel())

df["theta_alpha"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[8:16, 4:8].ravel())
df["delta_alpha"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[8:16, :4].ravel())

df["delta_theta"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[4:8, :4].ravel())

In [ ]:
rois = ['beta_gamma', 'alpha_gamma',
       'theta_gamma', 'delta_gamma', 'alpha_beta', 'theta_beta', 'delta_beta',
       'theta_alpha', 'delta_alpha', 'delta_theta']

In [ ]:
max_length = 182

df["beta_gamma"] = df["beta_gamma"].apply(lambda rec: resample(rec, max_length))
df["alpha_gamma"] = df["alpha_gamma"].apply(lambda rec: resample(rec, max_length))
df["theta_gamma"] = df["theta_gamma"].apply(lambda rec: resample(rec, max_length))
df["delta_gamma"] = df["delta_gamma"].apply(lambda rec: resample(rec, max_length))

df["alpha_beta"] = df["alpha_beta"].apply(lambda rec: resample(rec, max_length))
df["theta_beta"] = df["theta_beta"].apply(lambda rec: resample(rec, max_length))
df["delta_beta"] = df["delta_beta"].apply(lambda rec: resample(rec, max_length))

df["theta_alpha"] = df["theta_alpha"].apply(lambda rec: resample(rec, max_length))
df["delta_alpha"] = df["delta_alpha"].apply(lambda rec: resample(rec, max_length))

df["delta_theta"] = df["delta_theta"].apply(lambda rec: resample(rec, max_length))

In [ ]:
print(df[df["medication"] == "hcoffmed"]["subject"].unique())

In [ ]:
print(df[df["medication"] == "pdoffmed"]["subject"].unique())

In [ ]:
print(df[df["medication"] == "pdonmed"]["subject"].unique())

In [ ]:
data_corr = []
run_id = 0

for med in df.medication.unique():
    df_sel = df[df["medication"] == med]
    for sbj in df_sel.subject.unique():
        df_sbj = df_sel[df_sel["subject"] == sbj]
        for stim in df_sbj.stimulus.unique():
            df_stim = df_sbj[df_sbj["stimulus"] == stim]
            for channel in df_stim.channel.unique():
                df_chnl = df_stim[df_stim["channel"] == channel]
                for t1, t2 in combinations_with_replacement(df_chnl.trial.unique(), 2):
                    for reg1, reg2 in combinations_with_replacement(rois, 2):
                        x = df_chnl[(df_chnl["trial"] == t1)][reg1].to_numpy()[0]
                        y = df_chnl[(df_chnl["trial"] == t2)][reg2].to_numpy()[0]
                        corr_ = pearsonr(x, y)[0]
                        stats = {
                                "condition": med,
                                "stimulus": stim,
                                "subject": sbj,
                                "channel": channel,
                                "trial_1": t1,
                                "trial_2": t2,
                                "roi_1": reg1,
                                "roi_2": reg2,
                                "pac_1": x,
                                "pac_2": y,
                                "correlation": corr_
                                }
                        data_corr.append(stats)                        
                        run_id = run_id + 1
                        # print('Run id: ', run_id) ---> Wow!!!! increases running time!
                        # clear_output(wait=True) ---> Wow!!!! increases running time!
            print('Run id: ', run_id)

In [ ]:
df_corr = pd.DataFrame.from_records(data_corr)
print(df_corr.info())
print(df_corr)

In [ ]:
path_results = "/path-to-results/"
df_corr.to_csv(path_results + "pacs_corr.csv", index=False)